<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week7.do.txt --no_mako -->
<!-- dom:TITLE: March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# March 6-10, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University

Date: **March 6-10**

## Algorithms for solving quantum mechanical problems, quantum gates and circuits

1. Introduction to [Qiskit](https://qiskit.org/)

2. Defining one-, two- and three-qubit gates

3. Setting up quantum circuits and simple algorithms

4. VQE, Variational Quantum Eigensolver

5. Selected Hamiltonians

## Quantum gates, reminder from last week

## Introduction to [Qiskit](https://qiskit.org/)

In [1]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import qiskit as qk
from scipy.optimize import minimize


# # Initialize registers and circuit

n_qubits = 1 #Number of qubits
n_cbits = 1 #Number of classical bits (the number of qubits you want to measure at the end of the circuit)
qreg = qk.QuantumRegister(n_qubits) #Create a quantum register
creg = qk.ClassicalRegister(n_cbits) #Create a classical register
circuit = qk.QuantumCircuit(qreg,creg) #Create your quantum circuit


circuit.draw() #Draw circuit. It is empty

Thereafter we perform operations on qubit

In [2]:
circuit.x(qreg[0]) #Applies a Pauli X gate to the first qubit in the quantum register
circuit.draw()

and select  a qubit to measure and encode the results to a classical bit

In [3]:
#Measure the first qubit in the quantum register
#and encode the results to the first qubit in the classical register
circuit.measure(qreg[0],creg[0])
circuit.draw()

Thereafter we execute the circuit

In [4]:
backend = qk.Aer.get_backend('qasm_simulator') 
#This is the device you want to use. It is an ideal simulation of a quantum device


job = backend.run(circuit,shots=1000) #Run the circuit 1000 times
result = job.result()
counts = result.get_counts()
print(counts)
circuit.clear()
circuit.draw()


circuit.h(qreg[0]) #Apply a Hadamard gate to the first qubit of the quantum register
circuit.measure(qreg,creg)
print(circuit.draw())


job = backend.run(circuit,shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)
circuit.clear()

Now we create a two-qubit circuit and set up a Bell state

In [5]:
n_qubits = 2
n_cbits = 2
qreg = qk.QuantumRegister(n_qubits)
creg = qk.ClassicalRegister(n_cbits)
circuit = qk.QuantumCircuit(qreg,creg)
circuit.draw()



circuit.h(qreg[0])
circuit.cx(qreg[0],qreg[1]) 
#This is a controlled operation. Apply a Pauli X gate to the second qubit (qreg[1]) if the first qubit (qreg[0])
#is in the |1> state. Else do nothing

circuit.draw()

circuit.measure(qreg,creg)
circuit.draw()

In [6]:
job = backend.run(circuit,shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)
circuit.clear()

We apply a rotation to a qubit

In [7]:
theta = np.pi/3
circuit.rx(theta, qreg[0]) #R_x(theta) rotation on the first qubit (qreg[0])
circuit.measure(qreg,creg)
print(circuit.draw())
job = backend.run(circuit,shots=1000)
result = job.result()
counts = result.get_counts()
circuit.clear()
print(counts)

Now we want to find the lowest eigenvalue of $$ H = c_1 Z_0 + c_2 Z_1 + c_3 X_0 Y_1 $$ 
We will use $$<\psi|H|\psi> = c_1<\psi|Z_0|\psi> + c_2<\psi|Z_1|\psi> + c_3<\psi|X_0Y_1|\psi> $$

In [8]:
I = np.eye(2)
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
H = np.kron(Z,I) + np.kron(I,Z) + np.kron(X,Y)
eigvals,eigvecs = np.linalg.eigh(H)
print(eigvals[0])



c_1 = 1
c_2 = 1
c_3 = 1

h_1 = [c_1,[0],['z']]
h_2 = [c_2,[1],['z']]
h_3 = [c_3,[0,1],['x','y']]
H = [h_1,h_2,h_3]
H



H[0]

We create the ansatz

In [9]:
def ansatz(theta,n_qubits):
    qreg = qk.QuantumRegister(n_qubits)
    circuit = qk.QuantumCircuit(qreg)
    for i in range(n_qubits):
        circuit.ry(theta[i],qreg[i])
    for i in range(n_qubits-1):
        circuit.cx(qreg[i],qreg[i+1])
    return(circuit)
qreg = qk.QuantumRegister(n_qubits)
circuit = qk.QuantumCircuit(qreg)
circuit.h(qreg[:2])
print('Before ansatz')
print(circuit.draw())
theta = np.random.randn(2)
n_qubits = 2
circuit = circuit.compose(ansatz(theta,n_qubits))
print('After ansatz')
circuit.draw()

Now we change measurement basis

In [10]:
def basis_change(h_i,n_qubits):
    qreg = qk.QuantumRegister(n_qubits)
    circuit = qk.QuantumCircuit(qreg)
    
    for qubit,operator in zip(h_i[1],h_i[2]):
        if operator == 'x':
            circuit.h(qreg[qubit])
        if operator == 'y':
            circuit.sdg(qreg[qubit])
            circuit.h(qreg[qubit])
    return(circuit)
n_qubits = 2
qreg = qk.QuantumRegister(n_qubits)
circuit = qk.QuantumCircuit(qreg)
theta = np.random.randn(n_qubits)
circuit = circuit.compose(ansatz(theta,n_qubits))
print('Ansatz circuit')
circuit.draw()
circuit = circuit.compose(basis_change(H[2],n_qubits))
print('After basis transformation:')
print(circuit.draw())

Get energy for given rotational parameters $theta$

In [11]:
def get_energy(theta):
    n_qubits = 2
    qreg = qk.QuantumRegister(n_qubits)
    circuit = qk.QuantumCircuit(qreg)
    circuit = circuit.compose(ansatz(theta,n_qubits))
    circuit_list = []
    for idx,h_i in enumerate(H):
        basis_change_circuit = basis_change(h_i,n_qubits)
        new_circuit = circuit.compose(basis_change_circuit)
        creg = qk.ClassicalRegister(len(h_i[1]))
        new_circuit.add_register(creg)
        new_circuit.measure(qreg[h_i[1]],creg)
        circuit_list.append(new_circuit)
    shots = 10000
    job = backend.run(circuit_list,shots=shots)
    E = np.zeros(len(circuit_list))
    for i in range(len(circuit_list)):
        result = job.result()
        counts = result.get_counts(i)
        for key,value in counts.items():
            e = 1
            for bit in key:
                if bit == '0':
                    e *= 1
                if bit == '1':
                    e *= -1
            E[i] += e*value
        E[i] *= H[i][0]
    E /= shots
    return(np.sum(E))

theta = np.random.randn(2)
get_energy(theta)

Minimize energy with Scipy

In [12]:
theta = np.random.randn(2)
res = minimize(get_energy, theta, method='Powell',tol=1e-12)
get_energy(res.x)

We define  a more flexible ansatz

In [13]:
def ansatz(theta,n_qubits):
    qreg = qk.QuantumRegister(n_qubits)
    circuit = qk.QuantumCircuit(qreg)
    idx = 0
    for i in range(n_qubits):
        circuit.ry(theta[idx],qreg[i])
        idx += 1
    for i in range(n_qubits-1):
        circuit.cx(qreg[i],qreg[i+1])
    for i in range(n_qubits):
        circuit.rx(theta[idx],qreg[i])
        idx += 1
    for i in range(n_qubits-1):
        circuit.cx(qreg[i],qreg[i+1])
    return(circuit)
theta = np.random.randn(4)
res = minimize(get_energy, theta, method='Powell',tol=1e-16)
get_energy(res.x)

Minimize energy with gradient descent $$ \frac{\partial E (\theta_1,\dots,\theta_i,\dots,\theta_p)}{\partial \theta_i} = \frac{E(\theta_1,\dots,\theta_i + \pi/2,\dots, \theta_p) - E(\theta_1,\dots, \theta_i - \pi/2,\dots, \theta_p}{2} $$

In [14]:
epochs = 200
theta = np.random.randn(4)
for epoch in range(epochs):
    print(epoch,get_energy(theta))
    grad = np.zeros_like(theta)
    for idx in range(theta.shape[0]):
        theta_temp = theta.copy()
        theta_temp[idx] += np.pi/2
        E_plus = get_energy(theta_temp)
        theta_temp[idx] -= np.pi
        E_minus = get_energy(theta_temp)
        grad[idx] = (E_plus - E_minus)/2
    theta -= 0.1*grad

## Lipkin model

In [15]:
%matplotlib inline

import numpy as np
import qiskit
from qiskit.visualization import circuit_drawer
from qiskit.quantum_info import Statevector
from matplotlib.pyplot import figure
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, assemble
from qiskit.providers.aer.noise import NoiseModel
import pylatexenc
from qiskit.algorithms import VQE
from qiskit.utils import QuantumInstance
from qiskit.opflow import X, Z, I, Y
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import ADAM
from qiskit.opflow import AerPauliExpectation
from qiskit import IBMQ
import cmath
import pandas as pd
from scipy.sparse import diags
import numpy.linalg as LA
import matplotlib.pyplot as plt
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')
pi=np.pi

### Lipkin Model

The Lipkin Model (LM) was introduced by H. Lipkin, N. Meshkov, and A. Glick in 1965. The model considers two levels above and bellow the fermi level to which they are assigned an index of $\sigma_{\pm}$ each with degeneracy $\Omega$ over which fermions are distributed. Using the language of second quantunization representation, the LM

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
     \hat{H}_{LM} = \frac{1}{2}\epsilon\sum_{m\sigma}a^{\dagger}_{m\sigma}a_{m\sigma} -\frac{1}{2}V\sum_{mm'\sigma}a^{\dagger}_{m\sigma}a^{\dagger}_{m'\sigma}a_{m'-\sigma}a_{m-\sigma},
\label{_auto1} \tag{1}
\end{equation}
$$

where $\epsilon$ is the energy of the level, $V$ is the interaction
strength, and $m=1,\cdots,\Omega$. The first term is a one-body
interaction which counts the number of nucleons in the level. Where
the second term is a two-body interaction that destroys and created
nucleons up and down a level. One can introduce the quasi-spin
operators defined as

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
     \begin{split}
         \hat{K}_0 &= \frac{1}{2}\sum_{m=1}^{\Omega}(a^{\dagger}_{m+}a_{m+}-a^{\dagger}_{m-}a_{m-})\\ 
         \hat{K}_+ & = \sum_{m=1}^{\Omega}a^{\dagger}_{m+}a_{m-}\\
         \hat{K}_- & = (\hat{K}_+)^\dagger.
     \end{split}
\label{_auto2} \tag{2}
\end{equation}
$$

These operators have the following SU(2) angular momentum commutation relation

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}
     [\hat{K}_+,\hat{K}_-] =2\hat{K}_0, \hspace{1cm} [\hat{K}_0,\hat{K}_{\pm}]=\pm\hat{K}_{\pm}.
\label{_auto3} \tag{3}
\end{equation}
$$

The LM Hammiltonian can be written using the quasi-spin operators as such

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
     \hat{H}_{LM} = \epsilon\hat{K}_0 -\frac{1}{2}V(\hat{K}_+\hat{K}_++\hat{K}_-\hat{K}_-).
\label{_auto4} \tag{4}
\end{equation}
$$

The quasi-spin operators act like lowering and raising angular momentum operators, as such they have these given properties

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}
\hat{K}_0|KK_0\rangle = K_0|KK_0\rangle
\label{_auto5} \tag{5}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
    \hat{K}_{\pm}|KK_0\rangle =\sqrt{K(K+1)-K_0(K_0\pm 1)}|KK_0+1\rangle,
\label{_auto6} \tag{6}
\end{equation}
$$

With these properties we can calculate the LM Hamiltonian matrix

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation}
\langle KK_0|H_{LM}|KK_0'\rangle,
\label{_auto7} \tag{7}
\end{equation}
$$

where the non-zero elements are given by

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation}
\begin{split}
    \langle KK_0|H_{LM}|KK_0'\rangle & = \epsilon K_0\\
     \langle KK_0|H_{LM}|KK_0'\pm 2\rangle & = \langle KK_0\pm 2|H_{LM}|KK_0'\rangle \\
     &= -\frac{1}{2}VC,
\end{split}
\label{_auto8} \tag{8}
\end{equation}
$$

where C is the Clebsch-Gordan Coefficients one gets when $K_{\pm}^2$ operates on the state $|KK_0\rangle$.
Using equation (8) one can calculate the exact solution to the Lipkin Model for any $\Omega$.

The collective space of the Hamiltonian is $\Omega+1=3$, and is given by

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}
\begin{pmatrix}-\epsilon & 0 & -V\\
 0&0&0\\
 -V&0&\epsilon
\end{pmatrix}
\label{_auto9} \tag{9}
\end{equation}
$$

In [16]:
#function that sorts eigenvalues with its eigenvectors in accending order
def eigen(A):
    eigenValues, eigenVectors = LA.eig(A)
    idx = np.argsort(eigenValues)
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors) 

#one body expectation value
def one_body(E,N):
  k = N/2
  m = np.arange(-k,k+1,1) # Since the collective space is Omega+1
  return E*np.diag(m) #return a matrix where its diagonal elemens are epsilon*K_0

#two body expectation value
def two_body(V,N):
  k = N/2
  m = np.arange(-k,k+1,1)
  left =np.zeros(len(m)-2,dtype=complex)
  right = np.zeros(len(m)-2,dtype=complex)
  diag = np.zeros(len(m),dtype=complex)
  for i in range(len(left)):
    CG = cmath.sqrt(k*(k+1)-(m[i]+2)*(m[i]+1))*cmath.sqrt(k*(k+1)-m[i]*(m[i]+1)) #calculate Clebsch-Gordan Coefficients 
    left[i] = CG
    right[i] = CG
  k = [left,diag,right]
  offset = [-2,0,2]
  return -0.5*V*diags(k,offset).toarray() #return a matrix where its off digonal elements are (1/2)V(K^2_+ + K^2_-)

#full expectation value
def quasi_spin(E,V,N):
  ob = one_body(E,N)
  tb = two_body(V,N)
  H = ob+tb 
  e,v = eigen(H) # find the eigenvalues of the Hamiltonian
  return e,H

#converts chi to V
def Vp(E,omega,chi):
  return (chi*E)/(omega-1)

In [17]:
#parameters
E = 1
chi = np.arange(0,2.1,0.1)
omega = 2

EV0 = []
EV1 = []
EV2 = []
Ham = []
for i in chi:
  v = Vp(E,omega,i)
  EigenV,H = quasi_spin(E,v,omega) #return eigenvalues and Hamiltonian
  Ham.append(H)
  EV0.append(EigenV[0])
  EV1.append(EigenV[1])
  EV2.append(EigenV[2])

In [18]:
matrix = pd.DataFrame(Ham[5].real)
print('Hamiltonian matrix')
matrix.head()

In [19]:
plt.plot(chi,EV0)
plt.plot(chi,EV1)
plt.plot(chi,EV2)
plt.xlabel('$\chi$')
plt.ylabel('Energy')
plt.title('$\Omega=2$ exact Lipkin Model')

### Hartree Fock

A variational method in quantum mechanics, starts with the Schrodinger equation where one minimizes the energy for a given trail wave function $|\psi\rangle$.

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation}
\delta\frac{\langle\psi |\hat{H}|\psi\rangle}{\langle\psi|\psi\rangle}=0
\label{_auto10} \tag{10}
\end{equation}
$$

The Hartree Fock (HF) method is a variational method that assumes that
the exact N-body wavefunction of the system can be approimated by a
single Slater determinant, where energy of the system is minimized and
the trial wave function is the particle number conserving coherent
product state. The SU(2) coherent states map a two-dimensional sphere
known as the bloch sphere onto the complex plane,

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation}
\tau = tan\frac{\theta}{2}e^{-i\phi}
\label{_auto11} \tag{11}
\end{equation}
$$

where the normalized coherent SU(2) state can be written in the language of LM:

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation}
|\tau\rangle = \frac{1}{(1+|\tau|^2)^{\Omega/2}}e^{\tau\hat{K}_+}|\frac{\Omega}{2},-\frac{\Omega}{2}\rangle,
\label{_auto12} \tag{12}
\end{equation}
$$

The $E_{g.s}$ of the LM given HF coherent states is:

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div>

$$
\begin{equation}
E(\tau) = \langle \tau|\hat{H}_{LM}|\tau\rangle.
\label{_auto13} \tag{13}
\end{equation}
$$

To calculate equation the last euation, we use generating functions for:

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation}
\begin{split}
\langle \tau|\hat{K}_{0}|\tau\rangle &= \frac{1}{2}\Omega\frac{|\tau|^2-1}{\tau|^2+1}\\
\langle \tau|\hat{K}_{\pm}^2|\tau\rangle &= \Omega\frac{\tau^*}{|\tau|^2+1}
\end{split}
\label{_auto14} \tag{14}
\end{equation}
$$

Then using the definition of $\tau$ in equation (10) we can write down the energy:

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation}
E(\tau) = -\frac{\epsilon}{2}\Omega(cos\theta+\frac{\chi}{2}sin^2\theta cos2\phi),
\label{_auto15} \tag{15}
\end{equation}
$$

where $\chi=\frac{V}{\epsilon}(\Omega-1)$. Minimizing $E(\tau)$ with respect to $\theta,\phi$, we obtain the ground state Hartree Fock energy

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation}
\begin{split}
E_{HF}  = -\frac{\Omega}{2}\epsilon &\hspace{1cm} \chi<1\\
E_{HF}  = -\frac{\Omega}{4}\epsilon(\chi+\frac{1}{\chi}) &\hspace{1cm} \chi\ge 1
\end{split}
\label{_auto16} \tag{16}
\end{equation}
$$

In [20]:
def E_HF(x,E,omega):
  if x < 1:
    EHF = -(omega/2)*E
  elif x >= 1:
    EHF = -(omega/4)*E*(x+(1/x))
  return EHF

In [21]:
epsilon = 1
Omega = 2

HF = []
exact = []
for x in chi:
    HF.append(E_HF(x,epsilon,omega).real)
    v = Vp(epsilon,omega,x)
    e,h = quasi_spin(epsilon,v,omega)
    exact.append(e[0].real)

In [22]:
plt.plot(chi,HF,label='HF')
plt.plot(chi,exact,label='exact')
plt.xlabel('$\chi$')
plt.ylabel('Energy')
plt.axvline(1, color = 'k', linestyle='--')
plt.title('$\Omega=2$ exact Lipkin Model')
plt.legend(bbox_to_anchor=(1.05, 1))

<!-- # <center> Quantum Circuit  </center> -->
<!-- To solve the LM on a quantum computer we have to solve Schrodinger's equation. -->
<!-- \begin{equation} -->
<!-- H|\psi(\tilde{\theta})\rangle = E|\psi(\tilde{\theta})\rangle\\ -->
<!-- \end{equation} -->
<!-- In \[[LM2;@LM1]](#LM2;@LM1), they take the LM Hamiltonian from equation (1) mapping the creation and annihilation operators to Pauli operators Z, X, Y using the Jordan-Wigner transformation. The Hamiltonian then takes the form of -->
<!-- \begin{equation} -->
<!-- H = \frac{\epsilon}{2}(Z_1+Z_1) - \frac{V}{2}(X_1X_2 - Y_1Y_2) -->
<!-- \end{equation} -->
<!-- Where the Pauli operators are the measurement basis which the quantum circuit is measured in. However, before measuring, we first have to prepare the initial state $|\psi(\tilde{\theta})\rangle$ for $\Omega=2$ of the LM circuit -->
<!-- \begin{equation} -->
<!-- |\psi(\tilde{\theta})\rangle = cos(\tilde{\theta})|00\rangle+sin(\tilde{\theta})|11\rangle -->
<!-- \end{equation} -->
<!-- Where $|00\rangle =|\uparrow\uparrow\rangle$, $|11\rangle=|\downarrow\downarrow\rangle$, and $\tilde{\theta}=\theta-\frac{\pi}{2}$. The LM circuit for $\Omega=2$ is: -->
<!--  -->
<!-- ![qc.JPG](attachment:qc.JPG) -->
<!--  -->
<!-- This type of problem resides in the pantheon of variational methods where we minimize $\tilde{\theta}$. This problem was coded using IBM's python library Qiskit. Since this is a variational problem, the use of Qiskit's Variational Quantum Eigensolver (VQE) was used to find the optimal $\tilde{\theta}$ that minimizes $\langle\psi(\tilde{\theta})|H|\psi(\tilde{\theta})\rangle$. -->

<!-- In[13]: -->

In [23]:
def LM_circuit():
    theta = Parameter('theta')
    QC = QuantumCircuit(2)
    QC.ry(2*(theta-np.pi/2),0)
    QC.cnot(0,1)
    return QC

In [24]:
QC = LM_circuit()
QC.draw(output='mpl')

<!-- # <center> VQE method 1  </center> -->
<!--  -->
<!-- For this method, we will be using Qiskit's VQE function, where we specifiy -->
<!--  -->
<!-- - Quantum circuit -->
<!-- -  Optimizer -->
<!-- -  Quantum instance (i.e. which backend). Here we will be using the "qasm_simulator" -->
<!-- -  Initial point (i.e. $\theta$ search space) -->
<!-- - Hamiltonian/measurement basis -->
<!--  -->
<!-- To define the Hamiltonian, we will use qiskit Pauli operator functions I,Z,X,Y -->
<!--  -->
<!-- In this method, for a given $\chi$, we will do a search over $\theta$ from $-\frac{\pi}{2}$ to $\frac{\pi}{2}$ and picking out the minimum energy value -->

<!-- In[15]: -->

<!-- sim1 min example -->
sim = Aer.get_backend('qasm_simulator')
adam =qiskit.algorithms.optimizers.ADAM(maxiter=10000) #optimizer for VQE
epsilon = 1
omega = 2
chi2 = [0.5,1]

t = np.arange(-pi/2,pi/2,0.05) #0.1 step size finishs pretty fast
energy = []
for x in chi2:
    ev = []
    v = Vp(epsilon,omega,x)
    for i in range(len(t)):
        H = 0.5 * epsilon * ( Z ^ I ) +             0.5 * epsilon * ( I ^ Z ) -             0.5 * v * ( X ^ X ) +             0.5 * v * ( Y ^ Y )

        vqe = VQE(ansatz=LM_circuit(),optimizer=adam,initial_point=[t[i]],quantum_instance=sim,expectation=AerPauliExpectation())
        result = vqe.compute_minimum_eigenvalue(H)
        ev.append(result.eigenvalue)
    energy.append(ev)

<!-- In[16]: -->

plt.plot(t,energy[0],label='$\chi=0.5$')
plt.plot(t,energy[1],label='$\chi=1$')

plt.xlabel('$\\theta$')
plt.ylabel('Ground State Energy')
plt.title('$E_{g.s}$ vs. $\\theta$')
plt.legend()

<!-- In[17]: -->

epsilon = 1
omega = 2
chi3 = np.arange(0,2.1,0.1)
t = np.arange(-pi/2,pi/2,0.1)
adam =qiskit.algorithms.optimizers.ADAM(maxiter=10000)
LM_sim = []
for x in chi3:
    v = Vp(epsilon,omega,x)
    ev = []
    for i in range(len(t)):
        H = 0.5 * epsilon * ( Z ^ I ) +            0.5 * epsilon * ( I ^ Z ) -            0.5 * v * ( X ^ X ) +            0.5 * v * ( Y ^ Y )

        vqe = VQE(ansatz=LM_circuit(),optimizer=adam,initial_point=[t[i]],quantum_instance=sim,expectation=AerPauliExpectation())
        result = vqe.compute_minimum_eigenvalue(H)

        ev.append(result.eigenvalue)
    LM_sim.append(min(ev))

<!-- In[18]: -->

plt.plot(chi,np.array(exact),label='$E_{exact}$',color='b')
plt.plot(chi,np.array(HF),label='$E_{HF}$',color='g')
plt.scatter(chi3,LM_sim,label='QC sim',color='cyan')
plt.axvline(1, color = 'k', linestyle='--')
plt.ylabel('$E_{g.s}$')
plt.xlabel('$\chi$')
plt.title(f'$\Omega=2$')
plt.legend()

<!-- Here we will be running the same code, but with a noise model of IBM Quantum Computer Manila. -->
<!--  -->
<!-- Note this code takes a lot longer to run, so maybe reduce range of $\chi$ and $t$ i.e. $\theta$. -->

<!-- In[19]: -->

IBMQ.load_account()
provider = IBMQ.get_provider()
IBMQC=provider.get_backend('ibmq_manila')
noise_model = NoiseModel.from_backend(IBMQC)
qi = QuantumInstance(backend=sim, noise_model=noise_model)

<!-- In[ ]: -->

epsilon = 1
omega = 2
chi4 = np.arange(0,2.1,0.1)
t = np.arange(-pi/2,pi/2,0.1)
adam =qiskit.algorithms.optimizers.ADAM(maxiter=10000)
LM_sim_noise = []
for x in chi3:
    v = Vp(epsilon,omega,x)
    ev = []
    for i in range(len(t)):
        H = 0.5 * epsilon * ( Z ^ I ) +            0.5 * epsilon * ( I ^ Z ) -            0.5 * v * ( X ^ X ) +            0.5 * v * ( Y ^ Y )

        vqe = VQE(ansatz=LM_circuit(),optimizer=adam,initial_point=[t[i]],quantum_instance=qi,expectation=AerPauliExpectation())
        result = vqe.compute_minimum_eigenvalue(H)

        ev.append(result.eigenvalue)
    LM_sim_noise.append(min(ev))

<!-- In[ ]: -->

plt.plot(chi,np.array(exact),label='$E_{exact}$',color='b')
plt.plot(chi,np.array(HF),label='$E_{HF}$',color='g')
plt.scatter(chi3,LM_sim,label='QC sim',color='cyan')
plt.scatter(chi4,LM_sim_noise,label='QC sim with noise',color='r')
plt.axvline(1, color = 'k', linestyle='--')
plt.ylabel('$E_{g.s}$')
plt.xlabel('$\chi$')
plt.title(f'$\Omega=2$')
plt.legend()

<!-- # <center> VQE method 2  </center> -->
<!--  -->
<!-- For this method, we will be using Qiskit's VQE function, where we specifiy -->
<!--  -->
<!-- - Quantum circuit -->
<!-- -  Optimizer -->
<!-- -  Quantum instance (i.e. which backend). Here we will be using the "statevector_simulator" -->
<!-- - Hamiltonian/measurement basis -->

<!-- In[ ]: -->

epsilon = 1
omega = 2
chi5 = np.arange(0,2.1,0.1)
adam =qiskit.algorithms.optimizers.ADAM(maxiter=10000)
LM_sim = []
for x in chi5:
    v = Vp(epsilon,omega,x)

    H = 0.5 * epsilon * ( Z ^ I ) +        0.5 * epsilon * ( I ^ Z ) -        0.5 * v * ( X ^ X ) +        0.5 * v * ( Y ^ Y )

    vqe = VQE(ansatz=LM_circuit(),optimizer=adam,quantum_instance=vec,expectation=AerPauliExpectation())
    result = vqe.compute_minimum_eigenvalue(H)
    LM_sim.append(result.eigenvalue)

<!-- In[ ]: -->

plt.plot(chi,np.array(exact),label='$E_{exact}$',color='b')
plt.plot(chi,np.array(HF),label='$E_{HF}$',color='g')
plt.scatter(chi5,LM_sim,label='QC state vec',color='cyan')
plt.axvline(1, color = 'k', linestyle='--')
plt.ylabel('$E_{g.s}$')
plt.xlabel('$\chi$')
plt.title('$\Omega=2$')
plt.legend()

<!-- One thing to note is that method 2 only works for simulation. Where method 1 can work using IBM quantum computers. However, method 2 does run faster then method 1. -->

<!-- In[ ]: -->

<!-- In[ ]: -->